# Secure API Key Encryption and Retrieval

![A majestic vault of radiant treasures and timeless heirlooms.](./code_illustration/encryption_retrieval.jpg)

This notebook aims to securely encrypt and store an API key using a user-provided password. Initially, the user is prompted to input their API key and a password. A unique encryption key is then derived from the password using a random salt and the PBKDF2HMAC method. The API key is encrypted using this derived key and saved to disk. Subsequently, the script offers functionality to decrypt and retrieve the original API key, requiring the user to provide the same password used for encryption. This method ensures the API key's secure storage, protecting it from unauthorized access.

In [ ]:
import os
import pandas as pd
import openai
import getpass
import base64
import time
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend

In [ ]:
# Function to generate a random salt
def generate_salt():
    return os.urandom(16)

def generate_fernet_key(password, salt):
    password = password.encode()
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password))
    return key

def save_encrypted_api_key():
    api_key = input("Enter your API key: ")
    password = getpass.getpass("Enter your password: ")

    # Generate a random and secure salt
    salt = generate_salt()

    # Derive the Fernet key from the password and salt
    key = generate_fernet_key(password, salt)

    # Encrypt the API key
    cipher_suite = Fernet(key)
    encrypted_api_key = cipher_suite.encrypt(api_key.encode())

    with open("api_key.enc", "wb") as file:
        file.write(encrypted_api_key)

    with open("salt.bin", "wb") as file:
        file.write(salt)

    print("API key encrypted and saved successfully.")
    
def get_decrypted_api_key(password):
    try:
        with open("api_key.enc", "rb") as file:
            encrypted_api_key = file.read()

        with open("salt.bin", "rb") as file:
            salt = file.read()

        # Derive the Fernet key from the password and base64-decode the salt
        key = generate_fernet_key(password, salt)

        cipher_suite = Fernet(key)
        decrypted_api_key = cipher_suite.decrypt(encrypted_api_key).decode()

        return decrypted_api_key
    except FileNotFoundError:
        print("API key file not found.")
        return None
    except Exception as e:
        print("Error occurred while decrypting the API key:", str(e))
        return None



In [ ]:
#save_encrypted_api_key()

In [ ]:
# Prompt user for the password again to decrypt the API key
#password_for_decryption = getpass.getpass("Enter your password to decrypt the API key: ")

# Retrieve and decrypt the API key
#api_key = get_decrypted_api_key(password_for_decryption)